<a href="https://www.kaggle.com/code/wafair43/transformer-for-mt?scriptVersionId=216752989" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Attention Is All You Need

## Introduction
In This notebook, we will implement the english to german model from the paper titled, "Attention Is All You Need" from scratch in TensorFlow. The structure is as follows:
- Training Data
- Validation Data
- Test Data
- Transformer Model
- Loss
- Learning rate schedule
- Training


In [1]:
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Embedding, Dropout, Softmax, Masking
from tensorflow.keras.models import Model, Sequential
import tensorflow as tf
import numpy as np
import re
from tensorflow.keras.callbacks import Callback,ModelCheckpoint,CSVLogger,History
import time
import shutil
import os
import matplotlib.pyplot as plt
import numpy as np

2025-01-09 05:44:37.992418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 05:44:37.992561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 05:44:38.127601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
os.makedirs("/kaggle/working/data", exist_ok=True)

In [4]:

source_dir = "/kaggle/input/transformer-en-de-nmt-wmt14-bpe/"
destination_dir = "/kaggle/working/data/transformer-en-de-nmt-wmt14-bpe/"

if not os.path.exists(destination_dir):
    shutil.copytree(source_dir, destination_dir)
    print(f"Directory copied to: {destination_dir}")
else:
    print(f"Directory already exists: {destination_dir}")

Directory copied to: /kaggle/working/data/transformer-en-de-nmt-wmt14-bpe/


## Training Data 
### Background:
- We are implement WMT 2014 English-German NMT task.
- In the section 5 (Training) of the paper, it mentions:
- - ~ 4.5 million sentence pairs
- - Sentence were encoded using byte-pair encoding.
- - Has shared source-target vocabulary.
- - The vocabulary size is approx 37000.
- - Sentence pairs were batched together by approximate sequence length. Each training
batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000
target tokens.

#### Load Vocabulary

In [5]:
vocabulary = None
pattern = r'\n$'
root = "/kaggle/working/data/transformer-en-de-nmt-wmt14-bpe/"
with open(root+"vocab.bpe.32000", encoding='utf8') as h:
    vocabulary = h.readlines()
    vocabulary = list(map(lambda word:re.sub(pattern,'', word), vocabulary))
len(vocabulary)

special_tokens = ['<UNK>','<StartOfSequence>','<EndOfSequence>']
total_vocab = vocabulary + special_tokens
len(total_vocab)

36711

#### Create a Dictionary Mapping for the vocabulary

In [6]:
idx   = tf.range(1,len(total_vocab)+1) # start with 1, as 0 is for padding. Has total size of total_vocab
words = total_vocab

idx   = tf.constant(idx)
words = tf.constant(words)

words_idx_init = tf.lookup.KeyValueTensorInitializer(words, idx)
words_idx = tf.lookup.StaticHashTable(words_idx_init,-1)

idx_words_init = tf.lookup.KeyValueTensorInitializer(idx, words)
idx_words = tf.lookup.StaticHashTable(idx_words_init,'<UNK>')

In [7]:
max_seq_len = 64
vocab_size = len(total_vocab)
batch_size=128

#### Data generator

In [8]:
# Training data preprocessing
def preprocess_data(x,y):
   
    
    # Preprocess features
    
    e,d=x
    
    # split sentence
    e= tf.strings.split(tf.strings.join([start,e,end],separator=' ')) 
    
    # tokenize (convert to numbers)
    e = words_idx.lookup(e) 
    
    # Truncate sequence
    e = e[:max_seq_len] 
    
    #padding
    e = tf.concat([e, tf.zeros(max_seq_len-len(e), dtype="int32")],axis=0) 
    
    
    # split sentence
    d = tf.strings.split(tf.strings.join([start,d,end],separator=' ')) 
    
    # tokenize
    d = words_idx.lookup(d) 
    
    # Keep upto max_seq_len token
    d = d[:max_seq_len] 
    
    # Padding
    d = tf.concat([d, tf.zeros(max_seq_len-len(d), dtype="int32")],axis=0) 
    x = (e,d)

    # Preprocess labels
    
    y = tf.strings.split(tf.strings.join([y,end],separator=' '))
    y = words_idx.lookup(y)
    y = y[:max_seq_len]
    y = tf.one_hot(y,len(total_vocab)+1)
    
    #padding
    y = tf.concat([y, tf.zeros((max_seq_len-len(y),len(total_vocab)+1))],axis=0)
    
    
    
    return (x,y)

In [9]:
start = "<StartOfSequence>"
end   = "<EndOfSequence>"


# Dataset object for reading source english corpus (Encoder input)
dataset_en_train = tf.data.TextLineDataset(root+"train.tok.clean.bpe.32000.en")


# Dataset object for reading target german corpus (Decoder Input)
dataset_de_train = tf.data.TextLineDataset(root+"train.tok.clean.bpe.32000.de")

# Zipping the the features sources
train_X = tf.data.Dataset.zip((dataset_en_train,dataset_de_train))

# Dataset object for reading target german corpus (Label)
dataset_label_train = tf.data.TextLineDataset(root+"train.tok.clean.bpe.32000.de")

# Zipping features and labels
train = tf.data.Dataset.zip((train_X,dataset_label_train))

# Shuffle
train=train.shuffle(4700000)# 4462251 sentences in the dataset

# Apply Preprocessing
train=train.map(preprocess_data)

# Batching
train=train.batch(128)
train = train.prefetch(2)

## Validation Data
#### Data generator

In [10]:
start = "<StartOfSequence>"
end   = "<EndOfSequence>"


# Dataset object for reading source english corpus
dataset_en_val = tf.data.TextLineDataset([root+"newstest2012.tok.clean.bpe.32000.en",root+"newstest2013.tok.clean.bpe.32000.en" ])
dataset_en_val = dataset_en_val.map(lambda x: tf.strings.split(tf.strings.join([start,x,end],separator=' ')))
dataset_en_val = dataset_en_val.map(lambda x: words_idx.lookup(x))
dataset_en_val = dataset_en_val.map(lambda x: x[:max_seq_len])
dataset_en_val = dataset_en_val.padded_batch(batch_size,max_seq_len)

# Dataset object for reading target german corpus
dataset_de_val = tf.data.TextLineDataset([root+"newstest2012.tok.clean.bpe.32000.de",root+"newstest2013.tok.clean.bpe.32000.de" ])
dataset_de_val = dataset_de_val.map(lambda x: tf.strings.split(tf.strings.join([start,x,end],separator=' ')))
dataset_de_val = dataset_de_val.map(lambda x: words_idx.lookup(x))
dataset_de_val = dataset_de_val.map(lambda x: x[:max_seq_len])
dataset_de_val = dataset_de_val.padded_batch(batch_size,max_seq_len)


val_X = tf.data.Dataset.zip((dataset_en_val,dataset_de_val))

In [11]:
# Dataset object for reading target german corpus
dataset_label_val = tf.data.TextLineDataset([root+"newstest2012.tok.clean.bpe.32000.de",root+"newstest2013.tok.clean.bpe.32000.de" ])
dataset_label_val = dataset_label_val.map(lambda x: tf.strings.split(tf.strings.join([x,end],separator=' ')))
dataset_label_val = dataset_label_val.map(lambda x: words_idx.lookup(x))
dataset_label_val = dataset_label_val.map(lambda x: x[:max_seq_len]) # -1 because the label doesn't have the start token. I undid this
dataset_label_val = dataset_label_val.map(lambda x: tf.one_hot(x,len(total_vocab)+1)) #  +1 because idx ranges from idx 1 to idx len(total_vocab), but one shot always have index zero to len(total_vocab)
dataset_label_val = dataset_label_val.padded_batch(batch_size,(max_seq_len,len(total_vocab)+1))

In [12]:
val = tf.data.Dataset.zip((val_X,dataset_label_val))

## Test Data
#### Data Generator

In [13]:
start = "<StartOfSequence>"
end   = "<EndOfSequence>"


# Dataset object for reading source english corpus
dataset_en_test = tf.data.TextLineDataset([root+"newstest2014.tok.clean.bpe.32000.en",])
dataset_en_test = dataset_en_test.map(lambda x: tf.strings.split(tf.strings.join([start,x,end],separator=' ')))
dataset_en_test = dataset_en_test.map(lambda x: words_idx.lookup(x))
dataset_en_test = dataset_en_test.map(lambda x: x[:max_seq_len])
dataset_en_test = dataset_en_test.padded_batch(batch_size,max_seq_len)

# Dataset object for reading target german corpus
dataset_de_test = tf.data.TextLineDataset([root+"newstest2014.tok.clean.bpe.32000.de"])
dataset_de_test = dataset_de_test.map(lambda x: tf.strings.split(tf.strings.join([start,x,end],separator=' ')))
dataset_de_test = dataset_de_test.map(lambda x: words_idx.lookup(x))
dataset_de_test = dataset_de_test.map(lambda x: x[:max_seq_len])
dataset_de_test = dataset_de_test.padded_batch(batch_size,max_seq_len)


test_X = tf.data.Dataset.zip((dataset_en_test,dataset_de_test))

In [14]:
# Dataset object for reading target german corpus
dataset_label_test = tf.data.TextLineDataset([root+"newstest2014.tok.clean.bpe.32000.de"])
dataset_label_test = dataset_label_test.map(lambda x: tf.strings.split(tf.strings.join([x,end],separator=' ')))
dataset_label_test = dataset_label_test.map(lambda x: words_idx.lookup(x))
dataset_label_test = dataset_label_test.map(lambda x: x[:max_seq_len]) # -1 because the label doesn't have the start token. I undid this
dataset_label_test = dataset_label_test.map(lambda x: tf.one_hot(x,len(total_vocab)+1)) #  +1 because idx ranges from idx 1 to idx len(total_vocab), but one shot always have index zero to len(total_vocab)
dataset_label_test = dataset_label_test.padded_batch(batch_size,(max_seq_len,len(total_vocab)+1))

In [15]:
test = tf.data.Dataset.zip((test_X,dataset_label_test))

## The Transformer Model

We will be implementing the *base* model.

In [16]:
d_model = 512 # 
N = 6 # Number of stacks of Encoders and decoders
h = 8 # parallel attention layers
p_drop = 0.1
dff=2048 # first layer of the FFN

In [17]:
class ScaledDotProductAttention(Layer):
    def __init__(self,d_model, num_heads, **kwargs):
        super(ScaledDotProductAttention, self).__init__(**kwargs)
        self.supports_masking = True
        self.scaling_factor = tf.sqrt(d_model/num_heads)

   
        
    def call(self, Q,K,V, causal_mask=None, mask=None): 
        
    
        key_mask = K._keras_mask
        out = tf.matmul(Q,K, transpose_b=True) # matmul   
        out = tf.divide(out,self.scaling_factor) # scaling

       
        total_mask = None
        # Padding mask will never be none due to embedding layer always passing it  the mask
        mask_Q_num = tf.cast(mask, "float32")
        mask_K_num = tf.cast(key_mask, "float32")
        mask_QK = tf.matmul(mask_Q_num[...,None],mask_K_num[:,None])
        # Mask with illegal connections due to padding. Here, illegal connections are set True value
        illegal_padding_mask = tf.logical_not( tf.cast(mask_QK, dtype=tf.bool) )
        total_mask = illegal_padding_mask
         
    

          
        if causal_mask is not None: # this is bool mask with illegal connections set to True
            total_mask = tf.logical_or(causal_mask[None],total_mask)

        # Setting the illegal connections in the total mask to -infty to make them zero in the softmax computation
        out += tf.cast(total_mask, tf.float32) * tf.float32.min

        out = tf.nn.softmax(out,axis=-1)
        out = out * mask_Q_num[..., None]
        #print(out)
        
        
        out = tf.matmul(out,V)
        return out
            
            

### Multi-Head Attention
- This layer creates _h_ linear projections for Q, K, and V. Or _h_ dense layers for each (without bias term)
- Each linear projection projects the _d\_model_ (512 in this case) dimensional tokens in Q,K,and V to _d\_model/h_ dimensional tokens.
- Then it performs _h_ Scaled Dot-product Attention calculations on the _h_ tuples of lower dimensional projections of Q, K, and V and produce _h_ outputs.
- Finally, it contenates the _h_ outputs, and performs a final linear operation.

In [18]:
class MultiHeadAttention(Layer):

    """
    Notes: queries, keys, and values will be projected to learned linear projections of 
    dimension d_k, d_k, d_v
    """
    
    def __init__(self,d_model, num_heads, **kwargs):
        super(MultiHeadAttention, self).__init__( **kwargs)

        

        
        self.num_heads= num_heads
        self.scaledDotProductAttention = ScaledDotProductAttention(d_model, num_heads)
        
        self.W_Qs = []       # List of Query Weights matrices
        self.W_Ks = []       # List of Key Weights matrices
        self.W_Vs = []       # List of Values Weights matrices
        
        d_k = int(d_model/num_heads) # Projected key dimension
        d_v = d_k                    # Projected value dimension
        
        for i in range(num_heads): 
            self.W_Qs.append(Dense(units = d_k, use_bias=False))
            self.W_Ks.append(Dense(units = d_k, use_bias=False))
            self.W_Vs.append(Dense(units = d_v, use_bias=False))

        self.W_O =  Dense(units = d_model, use_bias=False) 

    def call(self, Q,K,V, causal_mask=None):
        attentionHeads = []
        for i in range(self.num_heads):

            # Project Querys, Keys and Values
            Q_i = self.W_Qs[i](Q) # Queries' Projection for the ith head
            K_i = self.W_Ks[i](K) # Keys' Projection for the ith head
            V_i = self.W_Vs[i](V) # Values' Projection for the ith head
            

            attentionHeads.append(self.scaledDotProductAttention(Q_i,K_i,V_i,causal_mask,mask=Q_i._keras_mask))

        # Concatenate all the attention heads on the last (feature) axis
        concat_heads = tf.concat(attentionHeads,axis=-1)

        # Final linear layer
        return self.W_O(concat_heads)



### FFN

In [19]:
class FFN(Layer):
    
    def __init__(self,dff, **kwargs):
        super(FFN, self).__init__()
        self.dense1 =  Dense(units=dff, activation='relu')
        self.dense2 =  Dense(units = d_model)

    
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return x

### Residual/Skip connections + Layer Norm

In [20]:
class ResidualBlock(Layer):
    def __init__(self, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.layerNorm = LayerNormalization()
        
    def call(self, sublayer_in, sub_layer_out, mask=None,):
        # masking input before addition. Remember pad values have non-zero vectors
        if mask is not None:
            numeric_mask = tf.cast(mask[...,None],tf.float32)
            masked_in = sublayer_in*numeric_mask
        else:
            masked_in = sublayer_in
            
        x = tf.add(sub_layer_out, masked_in) 
        x = self.layerNorm(x)
        return x

    

### Sublayer 1 & 2

In [21]:
class SubLayer1(Layer):
    
    def __init__(self, d_model, num_heads, p_drop, **kwargs):
        super(SubLayer1, self).__init__(**kwargs)

        self.supports_masking = True # Because SubLayer2 needs masking     
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.dropout = Dropout(rate=p_drop)
        self.resid = ResidualBlock()

    def call(self, Q,K,V,X, training=False, causal_mask=None, mask=None):
        
        x = self.mha(Q,K,V,causal_mask)
        x = self.dropout(x,training=training)

        x = self.resid(X,x,mask=mask)
        return x
         


In [22]:
class SubLayer2(Layer):
    def __init__(self,dff,p_drop, **kwargs):
        super(SubLayer2, self).__init__(**kwargs)
    
        self.ffn = FFN(dff)
        self.dropout = Dropout(rate=p_drop)
        self.resid = ResidualBlock()
        
    def call(self, X, training=False, mask=None ):
        
        x = self.ffn(X)
        x = self.dropout(x, training=training)

        x = self.resid(X,x,mask=mask)
        return x

### Encoder Layer


In [23]:
#### Encoder layer, not encoder
class EncoderLayer(Layer):

    
    def __init__(self, d_model, num_heads, dff, p_drop, **kwargs):
        super(EncoderLayer, self).__init__()
        self.supports_masking = True # for propogation
                        
        self.self_attention = SubLayer1(d_model, num_heads, p_drop)
        self.position_wise = SubLayer2(dff, p_drop)

    def call(self, encoder_input, mask=None, training=False):
        
        #print("encoder layer: " ,mask is not None)
        x = encoder_input
        x = self.self_attention(x,x,x,x,training=training, mask=mask)
        x = self.position_wise(x,training=training, mask=mask)
        return x
    

     

### Positional Embedding

In [24]:
def getPositionalEmbedding(seq_length=256, feature_dimension=None):


    # Returns featrure_dimension dimensional positional encodings for each position in a sequence of length seq_length.
    # return shape (sequ_length, feature_dimension)
    dimensions = tf.range(feature_dimension, dtype=tf.float32)
    positions = tf.range(seq_length, dtype=tf.float32)[...,None] # add additional dimension at the end for broadcasting
    even_dimensions = dimensions[::2]
    odd_dimensions   =  dimensions[1::2]

    feature_dimension = tf.cast(feature_dimension, tf.float32)
    even = tf.sin( positions/tf.pow(10000.,even_dimensions/feature_dimension)  )
    odd  = tf.cos( positions/tf.pow(10000.,odd_dimensions/feature_dimension) )

    # Since for a given position, concatenating the even and odd dimension is functinonally
    # equivalent to interleaving the even and odd dimensions, we will just stack them as follows. (I will later try interleave as well and check the time it takes) 

    sequence_position_embeddings = tf.concat([even,odd], axis=-1)
 
    return sequence_position_embeddings


In [25]:
class PositionalEncoding(Layer):
    """
    This layer takes ouput from the embedding layer. Embedding layer must have output shape of (batch, seq-len, features(d_model))
    """

    def __init__(self, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.supports_masking=True

    def call(self, inputs):
        
        # inputs shape should be (batch, seq-len, features(d_model))
        shape= tf.shape(inputs)
        seq_len, features = shape[1],shape[2]

        positionalEncodings = getPositionalEmbedding(seq_len, features)
        positionalEncodings = positionalEncodings[None,...] # add batch dimension along which to broadcast

        x = tf.add(inputs,positionalEncodings)  # This is equivalent to inputs+positionalEncodings[None]. 
                                                # Here, I explicitly added a batch dimension for broadcasting, but its not needed.
                                                # You can confirm as follows:  print(tf.math.reduce_all((inputs+positionalEncodings[None])==x))
        return x

        


### Decoder Layer 

In [26]:
class DecoderLayer(Layer):
    def __init__(self,d_model, num_heads,dff,p_drop, **kwargs):
        super(DecoderLayer, self).__init__()
        
        self.supports_masking=True # Ensures padding mask's propogation to the next decoder layer.
        self.masked_self_attention = SubLayer1(d_model, num_heads, p_drop) 
        self.encoder_decoder_attention = SubLayer1(d_model, num_heads, p_drop)
        self.position_wise = SubLayer2(dff, p_drop) # Position-wise feed forward

    # Important note: First are should be the tensor whose padding mask needs to be propagated because I chose the easiest way for now.
    def call(self,  decoder_input, encoder_output, causal_mask, mask=None, training=False):
        # tokenEmbeddings: Embeddings of tokens in a sentence
        #self attention, so Q, K, V are all same
        x = decoder_input
        x = self.masked_self_attention(x, x, x, x, causal_mask=causal_mask, mask=mask, training=training)
        x = self.encoder_decoder_attention(x,encoder_output,encoder_output,x, training=training, mask=mask)
        x = self.position_wise(x,training=training, mask=mask)
        return x
     

### Encoder

In [27]:
class Encoder(Layer):
    def __init__(self,N, d_model, num_heads,dff, p_drop, **kwargs):
        super(Encoder, self).__init__(**kwargs)

        self.encoder_layers = []
        for n in range(N):
            self.encoder_layers.append(EncoderLayer(d_model, num_heads,dff, p_drop))
    
    
    def call(self, inputs, training=False):
        x = inputs
        for i,layer in enumerate(self.encoder_layers):
            x = layer(x,training=training) # Each layer recieves masked tensor because the previous layer has self.supports_mask=True
            
        return x 
    


### Decoder

In [28]:
class Decoder(Layer):
    
    def __init__(self, N, d_model, num_heads, dff, seq_len, p_drop, **kwargs):
        super(Decoder, self).__init__()
        self.decoder_layers = []
        self.causal_mask = tf.constant(np.triu(np.ones((seq_len,seq_len)),k=1 ), dtype=tf.bool)

        for n in range(N):
            self.decoder_layers.append(DecoderLayer(d_model, num_heads,dff, p_drop))

    def call(self, enc_output,  dec_input, training=False):
        #def call(self, encoder_output, decoder_input, mask):
        x = dec_input
        for layer in self.decoder_layers:
            x= layer(x, enc_output,mask=x._keras_mask, causal_mask=self.causal_mask, training=training)
        return x
    

### Embedding scaling


In [29]:
class Scaling(Layer):
    # I created this layer, because simply doing a scalar multiplication with
    # a tensor with _keras_mask property resulted in a tensor with no mask.
    def __init__(self, scale, **kwargs):
        super(Scaling,self).__init__(**kwargs)
        self.supports_masking=True
        self.scale=scale
        
    def call(self, inputs):
        return inputs*self.scale
        


### Transformer

In [30]:
class Transformer(Model):
    def __init__(self, N, d_model, num_heads,dff,seq_len, vocab_size, p_drop, **kwargs):
        super(Transformer,self).__init__(**kwargs)
        self.shared_embedding = Embedding(vocab_size+1,d_model,mask_zero=True) # be becasue word indexes staart from 1, but embedding layer's embedding indexes start from 0.0 index is left for padding
        self.positional_encoding = PositionalEncoding()
        self.encoder = Encoder(N, d_model, num_heads,dff, p_drop)
        self.decoder =  Decoder(N, d_model, num_heads,dff,seq_len, p_drop)
        self.dropout1 = Dropout(p_drop)
        self.dropout2 = Dropout(p_drop)
        self.scale_embed =  Scaling(d_model**.5)

    def call(self, input, training=False):
        """
        enc_inputs : tokenized sequence of shape (batch, enc_seq_len)
        dec_inputs : tokenized sequence of shape (batch, dec_seq_len)
        """

        enc_inputs, dec_inputs = input
        
        enc_inputs = self.shared_embedding(enc_inputs)
        enc_inputs = self.scale_embed(enc_inputs)
        enc_inputs = self.positional_encoding(enc_inputs)  
        enc_inputs = self.dropout1(enc_inputs, training=training)
        
        dec_inputs = self.shared_embedding(dec_inputs)
        dec_inputs = self.scale_embed(dec_inputs)
        dec_inputs = self.positional_encoding(dec_inputs)
        dec_inputs = self.dropout2(dec_inputs, training=training)
        
        enc_output = self.encoder(enc_inputs, training=training)
        decoder_output = self.decoder(enc_output , dec_inputs,training=training) # has shape(batch, dec_input, d_model)

        # Final linear operation. Weight tying
        embed_weights = self.shared_embedding.weights[0] # has shape (vocab_size,d_model)
        embed_weights_scaled = self.scale_embed(embed_weights)

        logits = tf.matmul(decoder_output,embed_weights_scaled,transpose_b=True)

        return logits

## Loss
The label smoothing part.

In [31]:
loss_obj= tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1, reduction='none', from_logits=True)
mask_layer = Masking()

def loss_func(y_true, y_pred):
    
    lab_masked = mask_layer(y_true) # create mask from labels. If a particular token in a sequence has only zeros in the feature dimension, that token is masked as padding token.
    mask = tf.cast(lab_masked._keras_mask, tf.float32)
    loss = loss_obj(lab_masked[:,:,1:],y_pred[:,:,1:]) # The zeroth index of logits correspond to padding, so ignore it
    masked_loss = loss * mask # zero out the loss values corresponding to padding token vectors.
    
    return tf.reduce_sum(masked_loss)

### Learning rate schedule

In [32]:
class MyLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, warmup_steps=4000, d_model=512):
        super(MyLRSchedule,self).__init__()
        self.d_model = d_model#tf.cast(d_model, tf.float32)
        self.warmup_steps=warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        lrate = self.d_model**-.5 * tf.minimum(step**-0.5, step*self.warmup_steps**-1.5)
        return lrate

    
    
    def get_config(self):
        config = {
            'warmup_steps':self.warmup_steps,
            'd_model':self.d_model,
        }
        return config


In [33]:
transformer = Transformer(N, d_model, h,dff,max_seq_len, vocab_size, p_drop)

__Optimizer__

In [34]:
optimizer = tf.keras.optimizers.Adam(beta_1=.9,beta_2=.98,epsilon=10e-9, learning_rate=MyLRSchedule())
transformer.compile(optimizer=optimizer,loss=loss_func)

## Training
- Since kaggle has a session limit of 12 hours, model needs to be saved and reloaded to continue training.
- We ran one epoch per session. (Run training for one epoch, then renew the session, and continue training)
### Basic workflow
- For each epoch, respective folders needs to be created beforehand. The root folder was exp2, and inside that folder, there were folders 'epoch1', 'epoch2', 'epoch3'.
- Update the _epoch_ variable in the custom callback cell to ensure the files are saved in the epoch they are actually generated.
- If first epoch:
- - compile and fit.
- else: 
- - load the last save model, then compile and fit

__Note:__ Running the fit and validate method produces warnings regarding masks being lost. This is by design and shouldn't cause any worries. Masks aren't retained where they are not needed.

__Create folders needed for the current epoch__

In [35]:
#!rm -r /kaggle/working/exp2/epoch0

In [36]:

step_size = 7200 # 7200 batches before saving a checkpoint.
epoch='epoch0/'# Change this for each epoch
out_path='/kaggle/working/exp2/'+epoch
os.makedirs(out_path, exist_ok=True)
class CustomCallback(Callback):
    
    
    def on_batch_end(self,batch,logs):

        # Write batch number as well incase we want to resume from where we left of
        path = out_path+"batch.txt"
        f = open(path, "w")
        f.write(str(batch)) 
        f.write("\n")
        f.close()
        
        # Save model
        if (batch+1)%step_size==0:# +1 because zero (zeroth) batch is multiple of every number
            #training loss
            path = out_path+'train_loss.txt'
            f = open(path, "a")
            f.write(str(logs["loss"]))
            f.write("\n")
            f.close()

            
            # validation loss
            path = out_path+'val_loss.txt'
            f = open(path, "a")
            val_loss = self.model.evaluate(val)
            f.write(str(val_loss))
            f.write("\n")
            f.close()

            # time
            path = out_path+'time.txt'
            f = open(path, "a")
            f.write(time.strftime("%x %X"))
            f.write("\n")
            f.close()

            # Save model
            self.model.save_weights(root+'model_e_'+ str(round(val_loss,3))+'_.weights.h5')

            
            

    # Since the dataset is not exactly a multiple of steps, we need to consider the loss values when the epoch ends
    def on_epoch_end(self, epochs,logs):

        #training loss
            path = out_path+'train_loss.txt'
            f = open(path, "a")
            f.write(str(logs["loss"]))
            f.write("\n")
            f.close()

        
        # validation loss
            path = out_path+'val_loss.txt'
            f = open(path, "a")
            val_loss = self.model.evaluate(val)
            f.write(str(val_loss))
            f.write("\n")
            f.close()

            # time
            path = out_path+'time.txt'
            f = open(path, "a")
            f.write(time.strftime("%x %X"))
            f.write("\n")
            f.close()

            # Save model
            self.model.save_weights(root+'model_e_'+ str(round(val_loss,3))+'_.weights.h5')


In [37]:
transformer = Transformer(N, d_model, h,dff,max_seq_len, vocab_size, p_drop)
optimizer = tf.keras.optimizers.Adam(beta_1=.9,beta_2=.98,epsilon=10e-9, learning_rate=MyLRSchedule())
transformer.compile(optimizer=optimizer,loss=loss_func)

In [38]:
#uncomment to training
#history = transformer.fit(train,callbacks=[CustomCallback()])

#### continue training (Optional)
- restart session, load datasets, load all classes, modify _epoch_ in the callback so the current epoch's data saved in new folder.
- epochs 1,2,3,4,...

In [1]:
#transformer = Transformer(N, d_model, h,dff,max_seq_len, vocab_size, p_drop)
#optimizer = tf.keras.optimizers.Adam(beta_1=.9,beta_2=.98,epsilon=10e-9, learning_rate=MyLRSchedule())
#transformer.compile(optimizer=optimizer,loss=loss_func)

# Uncomment the beloe code to compile and model weights and optimizer's state 
# Comment if training the first epoch (already called fit in the above cell)

#transformer.fit(train.take(1))# This veryy important!!, otherwise the optimizer;s state is not loaded. The schedule depends on optimizer's state. very important!!
#weights_path = "/kaggle/input/transformer_attention/transformers/default/1/model_e_19466.451_.weights.h5" # latest weights from the last epoch. Change as per your environment
#transformer.load_weights(weights_path)

In [ ]:
#transformer.optimizer.iterations.numpy()# Double check if the optimizer's state is loaded correctly. This total number steps the optimizer has taken. Important for schedule

- Make sure to update callback code as mentioned above before running the following cell.

In [ ]:
# Uncomment to continue training
#history = transformer.fit(train,callbacks=[CustomCallback()])